# 100

In [1]:
import yaml
import numpy as np

set_name = "100"
ncwo = "ncwomax"

In [4]:
def get_MADs(data):
    MADs = []
    for set in data:
        ADs = []
        for system in data[set]:
            ADs.append(data[set][system]["AD"])
        MADs.append(np.mean(ADs))
    return MADs

def get_Sets(data):
    Sets = []
    for set in data:
        Sets.append(set)
    return Sets

def get_Descriptions(subsets):
    f_des = open('Sets_description.yaml', 'r')
    d_des = yaml.safe_load(f_des)
    descriptions = []
    for set in subsets:
        descriptions.append(d_des[set])
    return descriptions

In [5]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [6]:
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, TapTool
import math
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10

nof = "GNOF"
file = open('{}-{}-{}.yaml'.format(set_name,nof,ncwo), 'r')
data = yaml.safe_load(file)
subsets = get_Sets(data)
description = get_Descriptions(subsets)

nofs = ["PNOF5", "PNOF7s", "PNOF7", "GNOF", "GNOFm"]

data = {'subsets' : subsets}
for nof in nofs:
    if nof=="GNOFm":
        file = open('{}-{}-{}.yaml'.format(set_name,"GNOFc",ncwo), 'r')
    else:
        file = open('{}-{}-{}.yaml'.format(set_name,nof,ncwo), 'r')
    yaml_data = yaml.safe_load(file)
    counts = get_MADs(yaml_data)
    data[nof] = counts

descriptions = {'subsets' : subsets,
        'PNOF5'   : description,
        'PNOF7s'  : description,
        'PNOF7'   : description,
        'GNOF'    : description,
        'GNOFm'   : description}

x = [ (subset, nof) for subset in subsets for nof in nofs ]
counts = sum(zip(data['PNOF5'], data['PNOF7s'], data['PNOF7'], data['GNOF'], data['GNOFm']), ()) # like an hstack
descriptions = sum(zip(descriptions['PNOF5'], descriptions['PNOF7s'], descriptions['PNOF7'], descriptions['GNOF'], descriptions['GNOFm']), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts, descriptions=descriptions, nofs=[f for _,f in x]))

p = figure(x_range=FactorRange(*x), width=2000, height=350, title="100 Perfect Pairing")

p.vbar(x='x', top='counts', width=1.0, source=source, line_color="white", legend_field="nofs",
       fill_color=factor_cmap('x', palette=Category10[len(nofs)], factors=nofs, start=1, end=2))

p.y_range.start = 0
p.y_range.end = 50
p.xaxis.major_label_text_font_size = "0pt"
#p.xaxis.major_label_orientation = math.pi/2
p.xaxis.group_label_orientation = math.pi/2
p.xaxis.axis_label = "Sets"
p.yaxis.axis_label = "Errors (kcal/mol)"
p.xgrid.grid_line_color = None

p.legend.location = "top_center"
p.legend.orientation = "horizontal"

hover = HoverTool(tooltips=[
    ("Description", "@descriptions"),
    ("MAD", "@counts"),
])
p.add_tools(hover)

show(p)